In [1]:
classpath.add("org.apache.spark" % "spark-sql_2.11" % "1.6.1")
classpath.add("com.databricks" % "spark-csv_2.11" % "1.4.0")
classpath.add("org.apache.hadoop" % "hadoop-common" % "2.7.2")
classpath.add("com.github.wookietreiber" % "scala-chart_2.11" % "0.5.0")

Adding 141 artifact(s)
Adding 3 artifact(s)
Adding 13 artifact(s)
Adding 4 artifact(s)


In [2]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf

@transient val sparkConf = new SparkConf().setAppName("Ex3").setMaster("local[*]")
@transient val sc = new SparkContext(sparkConf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
16/05/18 22:41:54 INFO SparkContext: Running Spark version 1.6.1
16/05/18 22:41:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/18 22:41:54 INFO SecurityManager: Changing view acls to: qhuang
16/05/18 22:41:54 INFO SecurityManager: Changing modify acls to: qhuang
16/05/18 22:41:54 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users with view permissions: Set(qhuang); users with modify permissions: Set(qhuang)
16/05/18 22:41:56 INFO Utils: Successfully started service 'sparkDriver' on port 62769.
16/05/18 22:41:57 INFO Slf4jLogger: Slf4jLogger started
16/05/18 22:41:57 INFO Remoting: Starting remoting
16/05/18 22:41:57 INFO Remoting: Remoting started; listening on addresses :[akka.tcp://sparkDriverActorSystem@192.168.1.20:62782]
16/05/18 22:41:57 INFO Utils: Successfully started service '

import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
sparkConf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@4afab775
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@2337fba4

In [3]:
import org.apache.spark.sql.SQLContext

@transient val sqlContext = new SQLContext(sc)

import org.apache.spark.sql.SQLContext
sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@3dac0083

In [4]:
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.DataFrame
import org.jfree.data.category.DefaultCategoryDataset
import scalax.chart.module.ChartFactories

val searches = sqlContext.read
   .format("com.databricks.spark.csv")
   .option("header", "true")
   .option("delimiter", "^")
   .option("inferSchema", "true")
   .option("mode", "DROPMALFORMED")
   .load("dataset/searches.csv.bz2")
searches.registerTempTable("searchesTable")
val df = sqlContext.sql("select Date, Destination from searchesTable")
val resultAGP = monthlyCount(df, "AGP").collect
resultAGP.foreach(println)

val resultMAD = monthlyCount(df, "MAD").collect
resultMAD.foreach(println)

val resultBCN = monthlyCount(df, "BCN").collect
resultBCN.foreach(println)
createChart(resultAGP, resultMAD, resultBCN)


def monthlyCount(df: DataFrame, airport: String): RDD[(String, Long)] = {
    df.filter(col("Destination") === airport).map(r => (r.getString(0).substring(0, 7), 1L)).reduceByKey(_+_)
}

def createChart(resultAGP: Array[(String, Long)], resultMAD: Array[(String, Long)], resultBCN: Array[(String, Long)]) {
    val ds = new DefaultCategoryDataset
    resultAGP.foreach{ r =>
      ds.addValue(r._2, "AGP", r._1)
    }
    resultMAD.foreach{ r =>
      ds.addValue(r._2, "MAD", r._1)
    }
    resultBCN.foreach{ r =>
      ds.addValue(r._2, "BCN", r._1)
    }

    val chart = ChartFactories.BarChart(ds)
    chart.show()
}

: 

In [5]:
sc.stop()